<font color=red>**`I am using Google colab, since I'm using ubuntu 14.04 LTS which doesn't have tensorflow support and thus, I cannot use keras with it`**</font>


## Mounting google drive and changing directory

In [1]:
!pwd

/content


In [2]:
import os
os.getcwd()
os.listdir()

['.config', 'adc.json', 'drive', 'sample_data']

In [3]:
"""!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive"""

'!apt-get install -y -qq software-properties-common python-software-properties module-init-tools\n!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null\n!apt-get update -qq 2>&1 > /dev/null\n!apt-get -y install -qq google-drive-ocamlfuse fuse\nfrom google.colab import auth\nauth.authenticate_user()\nfrom oauth2client.client import GoogleCredentials\ncreds = GoogleCredentials.get_application_default()\nimport getpass\n!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL\nvcode = getpass.getpass()\n!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}\n\n!mkdir -p drive\n!google-drive-ocamlfuse drive'

In [4]:
os.listdir()
os.chdir('/content/drive/tensorflow')
print(os.getcwd())
print(os.listdir())

/content/drive/tensorflow
['fer2013-images.zip', 'fer2013.ipynb', 'logreg_coefficents.ipynb', 'Keras', 'fer2013', 'Youtube data classification.ipynb', 'Copy of basic-regression.ipynb', 'Copy of basic_classification.ipynb', 'Copy of basic-text-classification.ipynb', 'tensorflow_basics.ipynb', 'anova_fitting.ipynb', 'youtube_scraped.csv', 'images_fer2013']


## Data Preparation

In [5]:
import numpy as np
import pandas as pd

df = pd.read_csv('./youtube_scraped.csv')

df.head()

,Unnamed: 0,Video id,Title,Description,Category
0,0,e2NQE41J5eM,How do I travel so much ! How do I earn money!!,SUBSCRIBE - https://goo.gl/dEtSMJ ('MountainTr...,Travel and Blogs
1,1,i9E_Blai8vk,TRAVEL VLOG ∙ Welcome to Bali | PRISCILLA LEE,I had the chance to fly out to Bali with my wh...,Travel and Blogs
2,2,7ByoBJYXU0k,5 Steps to Becoming a Travel Blogger,"Travel blogger, Nikki Vargas, of The Pin the M...",Travel and Blogs
3,3,EthqIhPtd2I,"TRAVEL VLOG: SANTORINI, GREECE",Thank you so much for watching! I hope you fou...,Travel and Blogs
4,4,ehmsJLZlCZ0,Ep 1| Travelling through North East India | Of...,"The journey to Arunachal, North East India beg...",Travel and Blogs


In [6]:
print(df['Category'].unique())
df.dropna(inplace=True)

['Travel and Blogs' 'Science and Technology' 'Food' 'Manufacturing'
 'History' 'Art and Music']


### Label Encoding of the target label Category

In [7]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
df['targets'] = le.fit_transform(df['Category'])
df.head()

# targets has been created

,Unnamed: 0,Video id,Title,Description,Category,targets
0,0,e2NQE41J5eM,How do I travel so much ! How do I earn money!!,SUBSCRIBE - https://goo.gl/dEtSMJ ('MountainTr...,Travel and Blogs,5
1,1,i9E_Blai8vk,TRAVEL VLOG ∙ Welcome to Bali | PRISCILLA LEE,I had the chance to fly out to Bali with my wh...,Travel and Blogs,5
2,2,7ByoBJYXU0k,5 Steps to Becoming a Travel Blogger,"Travel blogger, Nikki Vargas, of The Pin the M...",Travel and Blogs,5
3,3,EthqIhPtd2I,"TRAVEL VLOG: SANTORINI, GREECE",Thank you so much for watching! I hope you fou...,Travel and Blogs,5
4,4,ehmsJLZlCZ0,Ep 1| Travelling through North East India | Of...,"The journey to Arunachal, North East India beg...",Travel and Blogs,5


In [8]:
list(le.classes_) # Thus, travel and blogs are given value of 5

['Art and Music',
 'Food',
 'History',
 'Manufacturing',
 'Science and Technology',
 'Travel and Blogs']

In [9]:
le.transform(['Travel and Blogs', 'Food'])

array([5, 1])

### Train Test split of the data

In [10]:
# all my categories have almost equal number of instances/samples

np.bincount(df['targets'])

array([1864, 2000, 2000, 1922, 1887, 1997])

In [0]:
from sklearn.model_selection import train_test_split

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['Description'],
                                                   df['targets'],
                                                   random_state=0)

In [12]:
print('X_train first entry:\n\n', X_train.iloc[0])
print('\n\nX_train shape: ', X_train.shape)

X_train first entry:

 Famous Marvel Cinematic Universe Information Leaks (Movie Spoilers) SUBSCRIBE NOW to CBR! Click here: https://goo.gl/wMuSDD In the age of the internet, ...


X_train shape:  (8752,)


### To one hot encoding of targets

In [0]:
def to_one_hot(labels, dimension=6):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1
        return results
    
one_hot_train_labels = to_one_hot(y_train)
one_hot_test_labels = to_one_hot(y_test)

In [14]:
one_hot_train_labels

array([[0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

## CountVectorizer

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

# Fit the CountVectorizer to the training data
vect = CountVectorizer().fit(X_train)

In [16]:
vect.get_feature_names()[::200] #Feature names

['00',
 'appointed',
 'buzybeez',
 'culture',
 'england',
 'gibt',
 'inside',
 'mains',
 'nostra',
 'prepper',
 'sayajigunj',
 'stressed',
 'types',
 'yoga']

In [17]:
len(vect.get_feature_names())

2688

In [18]:
# transform the documents in the training data to a document-term matrix
X_train_vectorized = vect.transform(X_train)

X_train_vectorized

<8752x2688 sparse matrix of type '<class 'numpy.int64'>'
	with 174865 stored elements in Compressed Sparse Row format>

In [19]:
X_train_vectorized.shape

(8752, 2688)

## Support Vector Machines (SVM)

In [20]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

clf = SVC()
grid_values = {'gamma': [0.001, 0.01, 0.05, 0.1, 1, 10, 100]}

# default metric to optimize over grid parameters: accuracy
grid_clf_acc = GridSearchCV(clf, param_grid = grid_values)
grid_clf_acc.fit(X_train_vectorized, y_train) # This best parameter values are
# for countvectorizer and not for n-grams or tfidf
 

print('Grid best parameter (max. accuracy): ', grid_clf_acc.best_params_)
print('Grid best score (accuracy): ', grid_clf_acc.best_score_)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Grid best parameter (max. accuracy):  {'gamma': 0.05}
Grid best score (accuracy):  0.9993144424131627


In [21]:
model = SVC(gamma = 0.05)
model.fit(X_train_vectorized, y_train)

print('Training Accuracy:' , model.score(X_train_vectorized, y_train))
print('Test Accuracy {}'.format(model.score(vect.transform(X_test), y_test)))

#Both training and test accuracy are high, so model seems to be good.


Training Accuracy: 1.0
Test Accuracy 0.9996572995202193


## Tfidf - term frequency inverse weighting

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Fit the TfidfVectorizer to the training data specifiying a minimum document frequency of 5
vect = TfidfVectorizer(min_df=5).fit(X_train)
len(vect.get_feature_names())

2369

In [23]:
X_train_vectorized = vect.transform(X_train)

model = SVC(C= 10, gamma='auto')
model.fit(X_train_vectorized, y_train)

print('Training Accuracy:' , model.score(X_train_vectorized, y_train))
print('Test Accuracy {}'.format(model.score(vect.transform(X_test), y_test)))

# Accuracy decreases way low from above, so tfidf performs poorly.

Training Accuracy: 0.7142367458866545
Test Accuracy 0.6997943797121315


### Smallest and largest tfidf

In [24]:
feature_names = np.array(vect.get_feature_names())

sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()

print('Smallest tfidf:\n{}\n'.format(feature_names[sorted_tfidf_index[:10]]))
print('Largest tfidf: \n{}'.format(feature_names[sorted_tfidf_index[:-11:-1]]))

Smallest tfidf:
['awesome' 'shops' 'restaurants' 'cooks' 'uncle' 'cute' 'kids' 'pink'
 'gave' 'just']

Largest tfidf: 
['science' 'prague' 'group' 'village' 'against' 'cheese' 'steel' 'channel'
 'lead' 'production']


## n-grams

In [25]:
# Fit the CountVectorizer to the training data specifiying a minimum 
# document frequency of 5 and extracting 1-grams and 2-grams
vect = CountVectorizer(min_df=5, ngram_range=(1,2)).fit(X_train)

X_train_vectorized = vect.transform(X_train)

len(vect.get_feature_names())

7618

In [26]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

clf = SVC()
grid_values = {'gamma': [0.001, 0.01, 0.05, 0.1, 1, 10, 100]}

# default metric to optimize over grid parameters: accuracy
grid_clf_acc = GridSearchCV(clf, param_grid = grid_values)
grid_clf_acc.fit(X_train_vectorized, y_train) # For n-grams
 

print('Grid best parameter (max. accuracy): ', grid_clf_acc.best_params_)
print('Grid best score (accuracy): ', grid_clf_acc.best_score_)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Grid best parameter (max. accuracy):  {'gamma': 0.05}
Grid best score (accuracy):  0.9990859232175503


In [27]:
X_train_vectorized = vect.transform(X_train)

model = SVC(gamma=0.05)
model.fit(X_train_vectorized, y_train)

print('Training Accuracy:' , model.score(X_train_vectorized, y_train))
print('Test Accuracy {}'.format(model.score(vect.transform(X_test), y_test)))


Training Accuracy: 1.0
Test Accuracy 0.9996572995202193


## Precision, Recall, F1_Score

In [28]:
from sklearn.metrics import precision_recall_fscore_support

predictions = model.predict(vect.transform(X_test))
precision, recall, f_score, support = precision_recall_fscore_support(y_test, predictions )
precision_recall_fscore_support(y_test, predictions )
#Precision, Recall, F-score and Support for each class

(array([1.        , 1.        , 1.        , 1.        , 0.99785408,
        1.        ]),
 array([1.   , 1.   , 0.998, 1.   , 1.   , 1.   ]),
 array([1.        , 1.        , 0.998999  , 1.        , 0.99892589,
        1.        ]),
 array([463, 525, 500, 497, 465, 468]))

In [29]:
df1 = pd.DataFrame([le.classes_, precision, recall, f_score]).T
df1.rename(columns={ 0: 'Category', 1:'precision', 2:'recall', 3: 'f_score'}, inplace = True)
df1

,Category,precision,recall,f_score
0,Art and Music,1,1,1
1,Food,1,1,1
2,History,1,0.998,0.998999
3,Manufacturing,1,1,1
4,Science and Technology,0.997854,1,0.998926
5,Travel and Blogs,1,1,1


### XGBOOST instead of bagging models or shallow neural networks

In [30]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [31]:
from xgboost import XGBClassifier

model = XGBClassifier(max_depth=2)
model.fit(X_train_vectorized, y_train)

#Overfitting at the moment, have to do parameter tuning.



print('Training Accuracy:' , model.score(X_train_vectorized, y_train))
print('Test Accuracy {}'.format(model.score(vect.transform(X_test), y_test)))

Training Accuracy: 0.9969149908592322
Test Accuracy 0.9955448937628513


In [32]:
from sklearn.metrics import precision_recall_fscore_support

predictions = model.predict(vect.transform(X_test))
precision, recall, f_score, support = precision_recall_fscore_support(y_test, predictions )
precision_recall_fscore_support(y_test, predictions )
#Precision, Recall, F-score and Support for each class

(array([0.98301486, 0.9980916 , 0.99201597, 1.        , 1.        ,
        1.        ]),
 array([1.        , 0.99619048, 0.994     , 0.99798793, 0.98494624,
        1.        ]),
 array([0.99143469, 0.99714013, 0.99300699, 0.99899295, 0.99241603,
        1.        ]),
 array([463, 525, 500, 497, 465, 468]))

In [33]:
df1 = pd.DataFrame([le.classes_, precision, recall, f_score]).T
df1.rename(columns={ 0: 'Category', 1:'precision', 2:'recall', 3: 'f_score'}, inplace = True)
df1

,Category,precision,recall,f_score
0,Art and Music,0.983015,1,0.991435
1,Food,0.998092,0.99619,0.99714
2,History,0.992016,0.994,0.993007
3,Manufacturing,1,0.997988,0.998993
4,Science and Technology,1,0.984946,0.992416
5,Travel and Blogs,1,1,1


### Using LSTM - Have some issues with the embedding layer will fix.

In [0]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df['Description'])

sequences = tokenizer.texts_to_sequences(df['Description'])
data = pad_sequences(sequences, maxlen=50)

In [35]:
model_lstm = Sequential()
model_lstm.add(Embedding(100, 100, input_length=50))
model_lstm.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model_lstm.add(Dense(6, activation='softmax'))
model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [36]:
model_lstm.fit(data, np.array(to_one_hot(df['targets'])), validation_split=0.4, epochs=10)

Instructions for updating:
Use tf.cast instead.
Train on 7002 samples, validate on 4668 samples
Epoch 1/10
7002/7002 [==============================] - 26s 4ms/step - loss: 2.5454e-04 - acc: 0.1148 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 2/10
7002/7002 [==============================] - 22s 3ms/step - loss: 2.1275e-04 - acc: 1.4282e-04 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 3/10
7002/7002 [==============================] - 22s 3ms/step - loss: 1.5631e-05 - acc: 1.4282e-04 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 4/10
7002/7002 [==============================] - 24s 3ms/step - loss: 9.6620e-09 - acc: 1.4282e-04 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 5/10
7002/7002 [==============================] - 22s 3ms/step - loss: 1.5391e-08 - acc: 1.4282e-04 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 6/10
7002/7002 [==============================] - 22s 3ms/step - loss: 1.0590e-08 - acc: 1.4282e-04 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Ep

### Using CNNs with LSTM

In [0]:
def create_conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(vocabulary_size, 100, input_length=50))
    model_conv.add(Dropout(0.2))
    model_conv.add(Conv1D(64, 5, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=4))
    model_conv.add(LSTM(100))
    model_conv.add(Dense(6, activation='softmax'))
    model_conv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_conv

In [38]:
model_conv = create_conv_model()
model_conv.fit(data, np.array(to_one_hot(df['targets'])), validation_split=0.4, epochs = 3)

Train on 7002 samples, validate on 4668 samples
Epoch 1/3
7002/7002 [==============================] - 9s 1ms/step - loss: 2.5540e-04 - acc: 0.1098 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 2/3
7002/7002 [==============================] - 7s 934us/step - loss: 1.9617e-04 - acc: 1.4282e-04 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 3/3
7002/7002 [==============================] - 6s 852us/step - loss: 3.5068e-07 - acc: 1.4282e-04 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
